In [ ]:
%run ./_utils

### Daily Snapshot — Awards
Exports awards entity in all 3 formats (JSONL, Parquet, Avro) with manifests.
Split from medium entities due to high daily volume (~11M records).

In [ ]:
from pyspark.sql import functions as F

date_str = get_snapshot_date()
print(f"Snapshot date: {date_str}")

df = get_daily_df(spark, "openalex.awards.awards_api", date_str)

# ID transform
df = df.withColumn("id", F.concat(F.lit("https://openalex.org/G"), F.col("id")))

# Drop funder_id
df = df.drop("funder_id")

# Coalesce null arrays to empty arrays
for col_name in ["investigators", "funded_outputs"]:
    df = df.withColumn(col_name, F.coalesce(F.col(col_name), F.array()))

# Export all formats
export_all_formats(spark, dbutils, df, date_str, "awards")

print(f"\nAwards export complete for {date_str}")